In [1]:
from read_edf import plot_signals
from read_result import read_result_file
from read_result import plot_sleep_stages
from split_signal import split_edf_by_annotations
from split_signal import split_edf_by_30s
from sklearn.decomposition import PCA
import processing_data
import os
import matplotlib.pyplot as plt
import numpy as np
from sklearn import svm
import svm_builder
import importlib

In [2]:
# edf_path = "../database/sleep-telemetry/ST7011J0-PSG.edf"
# result_path = "../database/sleep-telemetry/ST7011JP-Hypnogram.edf"

In [3]:


edf_files = []
result_files = []

for dirname, _, filenames in os.walk('../database/sleep-cassette'):
    for filename in filenames:
        if filename[9] == 'P':
            edf_files.append(os.path.join(dirname, filename))
        else:
            result_files.append(os.path.join(dirname, filename))
N = len(result_files)

X = np.empty((0,18000//processing_data.down_sampling_rate), float)
Y = np.empty((0,1), str)

print(N)

for i in range(N):
    if i % 10 == 0:
        print(f"{i}-th iteration!")
    result_path = result_files[i]
    edf_path = edf_files[i]
    # print(edf_path)
    # print(result_path)
    start_times, durations, labels = read_result_file(result_path)
    # print(durations)
    # print(labels)
    res = split_edf_by_annotations(edf_path, start_times, durations, labels)
    # print(res)
    split_signals = split_edf_by_30s(res)
    for (_, signals) in split_signals:
        for i in range(3):
            signals[i] = processing_data.signal_filter(signals[i])
    cur_X, cur_Y = processing_data.generate_matrix(split_signals)
    # print(cur_X.shape, cur_Y.shape)
    X = np.vstack((cur_X, X))
    Y = np.vstack((cur_Y, Y))
    # print(cur_X.shape)
    # print(cur_Y.shape)
    # break
print(X.shape)
print(Y.shape)

153
0-th iteration!
10-th iteration!
20-th iteration!
30-th iteration!
40-th iteration!
50-th iteration!
60-th iteration!
70-th iteration!
80-th iteration!
90-th iteration!
100-th iteration!
110-th iteration!
120-th iteration!
130-th iteration!
140-th iteration!
150-th iteration!
(14431, 1800)
(14431, 1)


In [4]:
pca1 = PCA(n_components= 200).fit(X)
pca2 = PCA(n_components= 0.8).fit(X)
X_pca1 = pca1.transform(X)
X_pca2 = pca2.transform(X)
print(X_pca1.shape)
print(X_pca2.shape)

(14431, 200)
(14431, 414)


In [15]:
# model = svm_builder.train_svm_model(X_pca1, Y)

model_x = X_pca1[:1200]
model_y = Y[:1200]
print(model_x.shape, model_y.shape)
# model_svm = svm_builder.train_svm_model(model_x, model_y)

(1200, 200) (1200, 1)


In [16]:
from bayes_builder import train_bayes_model
model_bayes = train_bayes_model(X_pca1, Y)

Training set score of Bayes: 0.45767745767745766
Testing set score of Bayes: 0.4579676674364896


In [17]:
model_svm = svm_builder.train_svm_model(model_x, model_y)

训练集： 0.9523809523809523
测试集： 0.24166666666666667
